In [255]:
import cv2
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sys

In [256]:
def find_connected_components(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY)[1]
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
    return num_labels, labels, stats, centroids,thresh

The find_connected_components(frame) function finds 8 connected components in the frame given in parameter

It performs pre-image processing to the frame converting it to gray scale, applying gaussian blur and then converting the frame to a binary image. The connectedComponentsWithStats() function inputs a binary image and returns (the number of components found, the x,y coordinates and the height and width(location of component)).While the component found is of irregular shape the stats provide us with a rectangle area of the frame in which component is located.

ISSUE:There is considerable data loss while converting Grayscale Image to Binary Image(components with lower intensity not detected)as accodring to task all components should be detected.FIX THRESHOLD VALUES.(???)

In [257]:
def draw_box(frame, stats):
    for label, stat in stats.items():
        x, y, w, h = stat[:4]  
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

The draw_box(frame, stats) function draws a bounding box around the components being tracked.

In [258]:
def plot_frequency_domain(component_intensity, fps):
    for label, intensity in component_intensity.items():
        n = len(intensity)
        T = 1 / fps
        yf = np.fft.fft(intensity)
        xf = np.linspace(0.0, 1.0/(2.0*T), n//2)
        plt.plot(xf, 2.0/n * np.abs(yf[:n//2]))
        plt.title(f'Frequency Domain for Component {label}')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.xticks(np.arange(0, max(xf) + 1, step=1))
        plt.show()

The plot_frequency_domain(component_intensity, fps) function takes the list component_intensity and plots the frquency domain graph of mean intencities of components in the 1 minute in which values have been recorded.

Frequency domain graphs show how the component_intensity values are distributed within different frequency bands over a range of frequencies.Therefore these plots could be used to figure out which components have their frequencies in the 0-1Hz band.(???)

In [259]:

def main(video):
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    tracked_components = defaultdict(dict) # saves the stats of the components being tracked 
    component_intensity = defaultdict(list)# saves the mean intensity of a component at each frame
    results = defaultdict(list)

    #loops through all the frames in the video
    for frame_index in range(total_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)  
        
        ret, frame = cap.read()
        if not ret:
            break

        #true when 1 minute has passed (512 frames) it calls the find_connected_components(frame) function and gets the components to be tracked for the next minute 
        #then the results that were tracked in the previous minute are printed.
        if frame_index % int(512) == 0:
            #connected compnents recalculated
            print(f"TOTAL NUMBER OF COMPONENTS THAT HAVE BEEN TRACKED {num_labels}:") if frame_index!=0 else print("")
            num_labels, labels, stats, _ ,t= find_connected_components(frame)
            k=0

            #printing the mean intensity of each component frame by frame
            for label, intensity in results.items():
                print(f"Results for component {label}:")
                for frame_index,l, m_intensity in intensity:
                    print(f"Frame {frame_index}: Intensity of Component {m_intensity}")

            # printing graph
            plot_frequency_domain(component_intensity, fps)

            # resets the lists and dictionaries
            tracked_components.clear()
            component_intensity.clear()
            results.clear()   

        #loops through all the components and finds the mean intensity of the component in that frame and append this value to the component_intensity list.
        for label in range(1, num_labels):
            x, y, w, h, _ = stats[label]  
            label_mask = (labels == label).astype(np.uint8) 
            mean_intensity = cv2.mean(frame, mask=label_mask)[0]
            tracked_components[label] = (x, y, w, h)
            component_intensity[label].append(mean_intensity)

        #appends all relevant information extracted from that frame to results so that it can be easily printed.
        for label, (x, y, w, h) in tracked_components.items():
                results[label].append((frame_index,label, component_intensity[label][k]))
        k+=1    
        
        # bounding boxes for components being tracked
        for label, (x, y, w, h) in tracked_components.items():
            draw_box(frame, {label: (x, y, w, h)})
                
        # video display
        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
             break

    
    cap.release()
    cv2.destroyAllWindows()


The main outer for loop loops through all the frames in the video

The if statement is true when 1 minute has passed it calls the find_connected_components(frame) function and gets the components to be tracked for the next minute then the results that were tracked in the previous minute are printed.

The for loop loops through all the components and finds the mean intensity of the component in that frame and append this value to the component_intensity list.

The for loop appends all relevant information extracted from that frame to results so that it can be easily printed.

In [ ]:

if __name__ == "__main__":
    video = '1705951007967.mp4'  
    main(video)
